Dev notes:
- through 2022/02/28, developed as "DataInventory_v2.5" or "DataInventory_v2 - Copy" in D:\ArcGIS\Projects\CM1 and Metadata Update
- from 2022/03/01, developed as (GitHub)/metadata/scripts/DataInventory

2022/03/01: v00.9

Setup & Requirements:
0. Requires ArcGIS Pro. Developed & tested at ArcGIS Pro 2.9.1 in the defualt python env. Run as Notebook (.ipynb) in ArcGIS Pro.
1. Acquire & stage data. Should be a copy of the most recent source data, not the source data itself. This process may involve changing geometry, attributes, and table schema.
2. Run "Check Geometry" tool. Use caution when deleting null geometry. Evaluate outputs and note necessary repairs.
3. **Run "Data Inventory" script tool (this Notebook). See below for next steps.**

## 1.  Inputs
Paste output file path into homefolder variable (retain r and quotes). Paste layer/file name into inputlayer variable.

In [ ]:
# Modify input names and paths here
homefolder = r''
inputlayer = r''

print('Inputs:\nFolder: {}\nLayer: {}\n\nReady.'.format(
    homefolder, inputlayer
    )
)

## 2. Automated Inventory
Run the remaining cell. Scroll down to see status messages.

In [ ]:
from os import path
import datetime
import openpyxl
from openpyxl import Workbook
from openpyxl.worksheet.datavalidation import DataValidation
from openpyxl.styles import Border, Side, Font, Alignment
from openpyxl.worksheet.table import Table, TableStyleInfo
from string import ascii_uppercase

homefolder = path.normpath(homefolder)
dateinventoried = datetime.datetime.now().isoformat()[:10]

print('Getting basic feature class info...')
desc = arcpy.Describe(inputlayer)
filename = ['Feature Class Information', desc.file]
dateinventoried2 = ['Date Inventoried', dateinventoried]
catalogpath = ['File Path', desc.catalogPath]
sr = ['Spatial Reference', desc.spatialReference.name]
featurecount = [
    'Number of Features', int(arcpy.management.GetCount(inputlayer)[0])
]
overview = [filename, dateinventoried2, catalogpath, sr, featurecount]

print('Getting detailed attribute and field info...')
# Numeric field types: 'SmallInteger', 'Integer', 'Single', 'Double'
flds = arcpy.ListFields(inputlayer)
countFields = len(flds)
headings = [
    'Field Name',  # 0
    'Alias',
    'Type',
    'Length',
    'Precision',
    'Scale',      # 5
    'Domain',
    'Default Value',
    'Editable',
    'Nullable',
    'Required',   # 10
    'Unique Values',
    'Nulls',
    'Spaces',
    'Blanks',
    'Longest String',  # 15
    'Minimum',
    'Maximum',
    'Keep or Delete',
    'Comments',
    'Definition',  # 20
    'Def. Source',
    'Domain Type',
    'Range Units',
    'Domain Notes'  # 24 (count: 25)
]

fArray = []
for f in flds:
    fAttr = [
        f.name,
        f.aliasName,
        f.type,
        f.length,
        f.precision,
        f.scale,
        f.domain,
        f.defaultValue
    ]  # 7
    if f.editable:
        fAttr.append('True')
    else:
        fAttr.append('False')  # 8
    if f.isNullable:
        fAttr.append('True')
    else:
        fAttr.append('False')  # 9
    if f.required:
        fAttr.append('True')
    else:
        fAttr.append('False')  # 10
    if f.type not in ['OID', 'Geometry']:
        with arcpy.da.SearchCursor(inputlayer, [f.name]) as c:
            records = [r[0] for r in c]
        fAttr.append(len(set(records)))  # 11
        fAttr.append(len([i for i in records if i is None]))  # 12
        fAttr.append(len([i for i in records if set(str(i)) == {' '}]))  # 13
        fAttr.append(len([i for i in records if i == '']))  # 14
    else:
        fAttr = fAttr + ['n/a']*4  # 11, 12, 13, 14
    if f.type == 'String' and [i for i in records if i]:
        fAttr.append(max([len(i) for i in records if i]))
    else:
        fAttr.append('n/a')  # 15
    if f.type in ['SmallInteger', 'Integer', 'Single', 'Double']:
        numonly = [i for i in records if i or i == 0]
        fAttr.append(min(numonly))
        fAttr.append(max(numonly))
    else:
        fAttr += ['n/a']*2  # 16, 17
    fArray.append(fAttr)
    if f.name.lower() in [
            'objectid', 'fid', 'shape', 'shape_length', 'shape_area'
            ]:
        fAttr = fAttr + ['n/a']*7

print('Setting up Excel workbook...')
wb = Workbook()
ws = wb.active

print('Writing results to Excel workbook...')
for lineitem in overview:
    ws.append(lineitem)
ws.append(['Fields'])
fieldRow = ws.max_row
ws.append(headings)
headRow = ws.max_row
for lineitem in fArray:
    ws.append(lineitem)

print('Formatting Excel workbook...')
columnDict = dict(zip(range(26), ascii_uppercase))

kd = columnDict[headings.index('Keep or Delete')]
dv = DataValidation(
    type='list',
    formula1='"Keep,Delete"',
    showDropDown=False,
    allow_blank=True
)
ws.add_data_validation(dv)
dv.add('{}{}:{}{}'.format(kd, headRow+1, kd, ws.max_row))

dt = columnDict[headings.index('Domain Type')]
dv = DataValidation(
    type='list',
    formula1='"Enumerated,Range,Codeset,Unrepresentable"',
    showDropDown=False,
    allow_blank=True
)
ws.add_data_validation(dv)
dv.add('{}{}:{}{}'.format(dt, headRow+1, dt, ws.max_row))

# '''
# schema = type::required
# stats = unique values::maximum
# maintenance = keep or delete::comments
# metadata = definition::domain notes
# '''

typeCol = columnDict[headings.index('Type')]
reqCol = columnDict[headings.index('Required')]
uniCol = columnDict[headings.index('Unique Values')]
maxiCol = columnDict[headings.index('Maximum')]
commCol = columnDict[headings.index('Comments')]
defCol = columnDict[headings.index('Definition')]
dn = columnDict[headings.index('Domain Notes')]

ws[f'{typeCol}{fieldRow}'] = 'Schema'
ws[f'{uniCol}{fieldRow}'] = 'Statistics'
ws[f'{kd}{fieldRow}'] = 'Maintenance'
ws[f'{defCol}{fieldRow}'] = 'Metadata'

double = Side(border_style='double')
medium = Side(border_style='medium')
thin = Side(border_style='thin')
bold13 = Font(size=13, bold=True)
bold12 = Font(size=12, bold=True)
bold11 = Font(size=11, bold=True)

for row in range(2, headRow):
    for col in range(ws.max_column):
        c = ws['{}{}'.format(columnDict[col], row)]
        c.border = Border(left=thin, right=thin, top=thin, bottom=thin)
        c.alignment = Alignment(horizontal='left')
        if col == 0:
            c.font = bold11

for col in range(ws.max_column):
    c = ws['{}1'.format(columnDict[col])]
    c.border = Border(left=thin, right=thin, bottom=double)
    c.font = bold13
    d = ws['{}{}'.format(columnDict[col], fieldRow)]
    d.border = Border(left=medium, right=medium)
    d.font = bold12
    d.alignment = Alignment(horizontal='center')

for row in range(1, fieldRow):
    ws.merge_cells('B{}:{}{}'.format(row, columnDict[ws.max_column-1], row))

ws.merge_cells(
    'A{}:B{}'.format(
        fieldRow,
        fieldRow
    )
)
ws.merge_cells(
    '{}{}:{}{}'.format(
        typeCol,
        fieldRow,
        reqCol,
        fieldRow
    )
)
ws.merge_cells(
    '{}{}:{}{}'.format(
        uniCol,
        fieldRow,
        maxiCol,
        fieldRow
    )
)
ws.merge_cells(
    '{}{}:{}{}'.format(
        kd,
        fieldRow,
        commCol,
        fieldRow
    )
)
ws.merge_cells(
    '{}{}:{}{}'.format(
        defCol,
        fieldRow,
        dn,
        fieldRow
    )
)

ws['A{}'.format(fieldRow)].font = bold13

fieldtable = Table(
    displayName='FieldTable',
    ref='A{}:{}{}'.format(headRow, columnDict[ws.max_column-1], ws.max_row)
)
style = TableStyleInfo(
    name='TableStyleLight15',
    showFirstColumn=False,
    showLastColumn=False,
    showRowStripes=True,
    showColumnStripes=False
)
fieldtable.tableStyleInfo = style
ws.add_table(fieldtable)

infowidth = max([len(i[0]) for i in overview])
fieldwidth = max([len(i[0]) for i in fArray])
aliaswidth = max([len(i[1]) for i in fArray])
headwidth = max([len(i) for i in headings])

ws.column_dimensions['A'].width = max(infowidth, fieldwidth)+1
ws.column_dimensions['B'].width = max(aliaswidth, headwidth)+2
for col in range(2, ws.max_column):
    ws.column_dimensions['{}'.format(columnDict[col])].width = headwidth+2

print('Saving Excel workbook to working folder...')
outfile = path.join(
    homefolder,
    '{}_{}.xlsx'.format(inputlayer, dateinventoried.replace('-', ''))
)
wb.save(outfile)

print('All done!')
